# SmoQy_Saver

Collect your outputs from SmoQyDQMC and do bin averaging across runs!

In [52]:
import numpy as np
import pandas as pd
import os
import csv

In [ ]:
Mu_i, Mu_f, Mu_step = -10.0, 10.0, 0.1
Beta_i, Beta_f, Beta_step = 1.0, 1.0, 1
U_i, U_f, U_step = 5.0, 5.0, 0.1
L_i, L_f, L_step = 6, 6, 1
sID_i, sID_f, sID_step = 1, 12, 1

Mus = np.arange(Mu_i, Mu_f+Mu_step, Mu_step).round(2)
Betas = np.arange(Beta_i, Beta_f+Beta_step, Beta_step).round(2)
Us = np.arange(U_i, U_f+U_step, U_step).round(2)
Ls = np.arange(L_i, L_f+L_step, L_step,dtype=int).round(2)
sIDs = np.arange(sID_i, sID_f+sID_step, sID_step).round(2)

sID = sIDs[0]
U = Us[0]
L = Ls[0]
mu = Mus[0]
beta = Betas[0]

N_bins = 3
Nperbin = len(sIDs) // N_bins
if len(sIDs) % N_bins != 0:
    raise ValueError("Number of sIDs is not divisible by N_bins")

lattice = 'kagome'
spaces = ['position', 'momentum']
time_displaced = False
integrated = False
LessIO = True # Use for LessIO SmoQy fork  
filepath = r"data" # Path to data directory

In [54]:
if lattice == 'square':
    N_bonds = 2
    N_orbitals = 1
elif lattice == 'kagome':
    N_bonds = 6
    N_orbitals = 3
else:
    raise ValueError(f"Unknown lattice type: {lattice}, manually define N_bonds and N_orbitals")  

In [55]:
def bins(data,Nperbin,Nbins):
    # Each bin is a point in the array, values are continuously added to it
    # Ex. 'Energy_bins' would be a 1xNbins array containing values of E (each value is a sum of Nperbin values)
    # This function takes the average of each bin
    
    # Nbins = len(data), but we can also supply it as an argument

    if Nbins != len(data):
        print('Check array size')
        return 
        
    if Nbins == 1 and Nperbin == 1:
        Bin_totalavg = data[0]
        ErrorBars = np.zeros_like(Bin_totalavg)
        return Bin_totalavg,ErrorBars

    # data = data.reshape(Nbins,Nperbin) # Reshape the data into a 2D array, where each row is a bin

    Bin_avgs = data / Nperbin # Not sure if this is correct but it seems roughly right? Don't really see where the math justification is though
    # You would think it should be Nperbin 

    Bin_totalavg=np.mean(Bin_avgs) #calculates one total value

    #This is where we calculate the error bars
    ErrorBars=0
    for i in range(Nbins):
        ErrorBars+=(Bin_avgs[i]-Bin_totalavg)**2
    ErrorBars=np.sqrt(1/Nbins)*np.sqrt(1/(Nbins-1))*np.sqrt(ErrorBars)

    return Bin_totalavg,ErrorBars

In [56]:
def get_stats(sID, U, mu, beta, L, filepath = f"data", lattice = "kagome", LessIO = False):
    '''
    Based on the naming parameters we supply sID, U, mu, and beta, lookup the folder name that the data from SmoQY is saved in.

    Paramters:
    sID (int): Simulation ID number
    U (float): Interaction Energy
    mu (float): Chemical Potential
    beta (float): Inverse Temperature 
    L (int): Number of times the unit cell is propogated along the unit vectors, for a 1D chain this is the number of sites
    filepath (string): Path to the parent folder where the data is stored
    
    Returns:
    global_stats (pd.dataframe): Global system statistics like density, action, compressibility, and double occupancy
    local_stats (pd.dataframe): Local statistics for each orbital like onsite energy, density, and double occupancy
    spin_z_corr (pd.dataframe): Spin z correlation statistics for equal-time measurements
    spin_x_corr (pd.dataframe): Spin x correlation statistics for equal-time measurements
    '''
    folder_name = f"hubbard_{lattice}_U{U:.2f}_mu{mu:.2f}_L{L}_b{beta:.2f}-{sID}"

    if LessIO:
        global_stats = pd.read_csv(f"{filepath}/{folder_name}/global_stats.csv", sep='\s+')
        local_stats = pd.read_csv(f"{filepath}/{folder_name}/local_stats.csv", sep='\s+')
        spin_z_corr = pd.read_csv(f"{filepath}/{folder_name}/spin_z_position_equal-time_stats.csv", sep='\s+')
        spin_x_corr = pd.read_csv(f"{filepath}/{folder_name}/spin_x_position_equal-time_stats.csv", sep='\s+')
    else:
        global_stats = pd.read_csv(f"{filepath}/{folder_name}/global_stats.csv", sep='\s+')
        local_stats = pd.read_csv(f"{filepath}/{folder_name}/local_stats.csv", sep='\s+')
        spin_z_corr = pd.read_csv(f"{filepath}/{folder_name}/equal-time/spin_z/spin_z_position_equal-time_stats.csv", sep='\s+')
        spin_x_corr = pd.read_csv(f"{filepath}/{folder_name}/equal-time/spin_x/spin_x_position_equal-time_stats.csv", sep='\s+')
    
    return global_stats, local_stats, spin_z_corr, spin_x_corr

In [57]:
def grab_data(stats, observable, data):
    '''
    Grabs the data for a given observable from the stats dataframe.
    
    Parameters:
    stats (pd.dataframe): Dataframe containing the statistics
    observable (str): Name of the observable to grab data for
    
    Returns:
    data (np.array): Array containing the data for the observable
    '''
    row = stats[stats["MEASUREMENT"] == observable]
    if not row.empty:
        values = row['MEAN_R'].values
        if len(values) == 1:
            return values[0]
        else:
            # check that the values are of the expected length
            if len(data[observable][0]) == len(values):
                return values
            else:
                print(f"Length mismatch for observable '{observable}': expected {len(data[observable])}, got {len(values)}")
                return None
    else:
        print(f"Observable '{observable}' not found in stats.")
        return None

In [58]:
def find_nearest_neighbors(spin_z_corr, spin_x_corr, lattice, correlation):

    if correlation == 'spin-z-position_equal-time':
        dataframe = spin_z_corr
    elif correlation == 'spin-x-position_equal-time':
        dataframe = spin_x_corr

    if lattice == 'square':
        nearest_neighbors = np.array((
            dataframe.loc[(dataframe['R_1'] == 0) & (dataframe['R_2'] == 1), 'MEAN_R'].values,
            dataframe.loc[(dataframe['R_2'] == 0) & (dataframe['R_1'] == 1), 'MEAN_R'].values
        ))

    elif lattice == 'kagome':

        within_unit_cell = np.array(((1,2,0,0),
                                    (1,3,0,0),
                                    (2,3,0,0)))

        nearest_neighbors = []
        
        ######## within unit cell ########
        for i in range(len(within_unit_cell)):
            nearest_neighbors.append(dataframe.loc[
                (dataframe['ORBITAL_ID_1'] == within_unit_cell[i][0]) &
                (dataframe['ORBITAL_ID_2'] == within_unit_cell[i][1]) &
                (dataframe['R_1'] == within_unit_cell[i][2]) &
                (dataframe['R_2'] == within_unit_cell[i][3]),
                'MEAN_R'
            ].values)

    return nearest_neighbors

In [ ]:
# Initialize data structures
data = {}
results = {}

global_observables = ['sgn', 'sgndetGup', 'sgndetGdn', 'density', 'density_up', 'density_dn', 'double_occ', 'Nsqrd', 'chemical_potential', 'compressibility']
local_observables = ['hubbard_energy', 'hopping_energy', 'onsite_energy']
if LessIO:
    global_observables += ['total_energy', 'total_energy_sqrd']

for observable in global_observables:
    data[observable] = np.zeros((len(Mus), N_bins))
for observable in local_observables:
    if observable in ['hubbard_energy', 'onsite_energy']:
        data[observable] = np.zeros((len(Mus), N_orbitals, N_bins))
    elif observable == 'hopping_energy':
        data[observable] = np.zeros((len(Mus), N_bonds, N_bins))

correlations = ['spin-z-position_equal-time', 'spin-x-position_equal-time', 'structure_factor']
corr_bins = {corr: np.zeros((len(Mus), N_bins)) for corr in correlations}

# Combined loop
for mu_index, mu in enumerate(Mus):
    mu_val = 0.0 if -0.05 < mu < 0.05 else mu
    for sID_index, sID in enumerate(sIDs):
        try:
            global_stats, local_stats, spin_z_corr, spin_x_corr = get_stats(sID, U, mu_val, beta, L, lattice=lattice, LessIO=LessIO, filepath=filepath)
            # Global observables
            for observable in global_observables:
                data[observable][mu_index][sID_index % N_bins] += grab_data(global_stats, observable, data)
            # Local observables
            for observable in local_observables:
                if len(data[observable][0]) == N_orbitals:
                    if N_orbitals == 1:
                        data[observable][mu_index][0][sID_index % N_bins] += grab_data(local_stats, observable, data)
                    else:
                        for orbital_index in range(N_orbitals):
                            data[observable][mu_index][orbital_index][sID_index % N_bins] += grab_data(local_stats, observable, data)[orbital_index]
                elif len(data[observable][0]) == N_bonds:
                    for bond_index in range(N_bonds):
                        data[observable][mu_index][bond_index][sID_index % N_bins] += grab_data(local_stats, observable, data)[bond_index]
            # Correlations
            for correlation in correlations:
                if correlation == "structure_factor":
                    total_value = np.sum((spin_z_corr['MEAN_R'].values + 2*spin_x_corr['MEAN_R'].values)/3)
                    corr_bins[correlation][mu_index][sID_index % N_bins] += total_value
                else:
                    nearest_neighbors = find_nearest_neighbors(spin_z_corr, spin_x_corr, lattice, correlation)
                    nearest_value = np.mean(nearest_neighbors)
                    corr_bins[correlation][mu_index][sID_index % N_bins] += nearest_value
        except Exception as e:
            print(f"Error retrieving data for sID: {sID}, U: {U}, mu: {mu_val}, beta: {beta}, L: {L}. Skipping this configuration.")

# Bin and store results
for observable in global_observables:
    results[observable] = np.zeros((2, len(Mus)))
    for i in range(len(Mus)):
        results[observable][0][i], results[observable][1][i] = bins(data[observable][i], Nperbin, N_bins)

for observable in local_observables:
    if len(data[observable][0]) == N_orbitals:
        results[observable] = np.zeros((N_orbitals, 2, len(Mus)))
        for orbital_index in range(N_orbitals):
            for i in range(len(Mus)):
                results[observable][orbital_index][0][i], results[observable][orbital_index][1][i] = bins(data[observable][i][orbital_index], Nperbin, N_bins)
    elif len(data[observable][0]) == N_bonds:
        results[observable] = np.zeros((N_bonds, 2, len(Mus)))
        for bond_index in range(N_bonds):
            for i in range(len(Mus)):
                results[observable][bond_index][0][i], results[observable][bond_index][1][i] = bins(data[observable][i][bond_index], Nperbin, N_bins)

for correlation in correlations:
    results[correlation] = np.zeros((2, len(Mus)))
    for i in range(len(Mus)):
        results[correlation][0][i], results[correlation][1][i] = bins(corr_bins[correlation][i], Nperbin, N_bins)

results['spin-xyz-position_equal-time'] = np.zeros((2, len(Mus)))
results['spin-xyz-position_equal-time'][0] = 1/3 * results['spin-z-position_equal-time'][0] + 2/3 * results['spin-x-position_equal-time'][0]
results['spin-xyz-position_equal-time'][1] = np.sqrt(
    (1/3 * results['spin-z-position_equal-time'][1])**2 +
    (2/3 * results['spin-x-position_equal-time'][1])**2
)

In [78]:
dont_write = ['spin-z-position_equal-time', 'spin-x-position_equal-time', 'hubbard_energy', 'hopping_energy', 'onsite_energy']

# Average and error for hubbard_energy
results['hubbard_energy_avg'] = np.zeros((2, len(Mus)))
for i in range(len(Mus)):
    results['hubbard_energy_avg'][0][i] = np.sum(results['hubbard_energy'][:, 0, i])
    results['hubbard_energy_avg'][1][i] = np.sqrt(np.sum(results['hubbard_energy'][:, 1, i] ** 2))

# Average and error for hopping_energy
results['hopping_energy_avg'] = np.zeros((2, len(Mus)))
for i in range(len(Mus)):
    results['hopping_energy_avg'][0][i] = np.sum(results['hopping_energy'][:, 0, i])
    results['hopping_energy_avg'][1][i] = np.sqrt(np.sum(results['hopping_energy'][:, 1, i] ** 2))

# Average and error for onsite_energy
results['onsite_energy_avg'] = np.zeros((2, len(Mus)))
for i in range(len(Mus)):
    results['onsite_energy_avg'][0][i] = np.sum(results['onsite_energy'][:, 0, i])
    results['onsite_energy_avg'][1][i] = np.sqrt(np.sum(results['onsite_energy'][:, 1, i] ** 2))

filename = 'SmoQy_Results.csv'
header = ['observable', 'mu', 'value', 'error', 'beta', 'L', 'U', 'num_runs', 'lattice']
num_runs = sIDs[-1] - sIDs[0] + 1

existing_entries = set()
if os.path.exists(filename):
    with open(filename, 'r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            key = (row['mu'], row['beta'], row['L'], row['U'], row['lattice'])
            existing_entries.add(key)

with open(filename, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write header only if file is new/empty
    if os.stat(filename).st_size == 0:
        writer.writerow(header)
    for key in results:
        if key in dont_write:
            continue
        if isinstance(results[key], np.ndarray) and results[key].ndim == 2 and results[key].shape[1] == len(Mus):
            for i, mu in enumerate(Mus):
                entry_key = (str(mu), str(beta), str(L), str(U), lattice)
                if entry_key not in existing_entries:
                    writer.writerow([key, mu, results[key][0][i], results[key][1][i], beta, L, U, num_runs, lattice])
        elif isinstance(results[key], np.ndarray) and results[key].ndim == 3 and results[key].shape[2] == len(Mus):
            for idx in range(results[key].shape[0]):
                for i, mu in enumerate(Mus):
                    entry_key = (str(mu), str(beta), str(L), str(U), lattice)
                    if entry_key not in existing_entries:
                        writer.writerow([f"{key}_{idx}", mu, results[key][idx][0][i], results[key][idx][1][i], beta, L, U, num_runs, lattice])
        else:
            print(f"Skipping key '{key}' with unexpected shape: {np.shape(results[key])}")